# 2010-2011 Performance of NYC High schools

This project I will extract, transform, and load three datasets into a local database.

Using Pandas, the datasets are extracted, cleaned, then loaded into three tables in the database. Then using sqlite, the tables are merged into one table, extracting unique attribues from each table. I then query the best schools from the table.

Datasets provide data on the performance of NYC High schools during the 2010-2011 school year and can be found at:
https://data.cityofnewyork.us/Education/2010-2011-School-Progress-Reports-All-Schools/yig9-9zum <br>
https://data.cityofnewyork.us/Education/2010-AP-College-Board-School-Level-Results/itfs-ms3e <br>
https://data.cityofnewyork.us/Education/2010-SAT-College-Board-School-Level-Results/zt9s-n5aj <br>

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
#Load the three datasets into a Pandas dataframe
df1 = pd.read_csv('2010_2011_School_Progress_Reports_All_Schools.csv')
df2 = pd.read_csv('2010_AP_College_Board_School_Level_Results.csv')
df3 = pd.read_csv('2010_SAT_College_Board_School_Level_Results.csv')

In [3]:
print(df1.head())

      DBN  DISTRICT                          SCHOOL              PRINCIPAL  \
0  01M015         1       P.S. 015 Roberto Clemente          Irene Sanchez   
1  01M019         1             P.S. 019 Asher Levy    Jacqueline Flanagan   
2  01M020         1            P.S. 020 Anna Silver              James Lee   
3  01M034         1  P.S. 034 Franklin D. Roosevelt  Joyce Stallings Harte   
4  01M063         1       P.S. 063 William McKinley     Darlene Despeignes   

  PROGRESS REPORT TYPE SCHOOL LEVEL*  PEER INDEX* 2010-2011 OVERALL GRADE  \
0                  EMS    Elementary        63.61                      C    
1                  EMS    Elementary        49.48                      B    
2                  EMS    Elementary        56.65                      B    
3                  EMS           K-8        67.97                      C    
4                  EMS    Elementary        58.85                      B    

   2010-2011 OVERALL SCORE  2010-2011 ENVIRONMENT CATEGORY SCORE  \


In [4]:
print(df2.head())

      DBN                             SchoolName AP Test Takers   \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.              39   
1  01M450                 EAST SIDE COMMUNITY HS              19   
2  01M515                    LOWER EASTSIDE PREP              24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH             255   
4  02M296  High School of Hospitality Management               s   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                    s  
2                26                                   24  
3               377                                  191  
4                 s                                    s  


In [5]:
print(df3.head())

      DBN                                     School Name  \
0  01M292  Henry Street School for International Studies    
1  01M448            University Neighborhood High School    
2  01M450                East Side Community High School    
3  01M458                   SATELLITE ACADEMY FORSYTH ST    
4  01M509                               CMSP HIGH SCHOOL    

  Number of Test Takers Critical Reading Mean Mathematics Mean Writing Mean  
0                    31                   391              425          385  
1                    60                   394              419          387  
2                    69                   418              431          402  
3                    26                   385              370          378  
4                     s                     s                s            s  


In [6]:
#Rename columns
df1 = df1.rename(columns={'SCHOOL': 'School_Name', '2010-2011 OVERALL GRADE': 'Overall_Grade',  '2009-10 PROGRESS REPORT GRADE' : 'Progress_Grade'})
df2 = df2.rename(columns={'SchoolName': 'School_Name'})
df3 = df3.rename(columns={'School Name': 'School_Name'})

In [7]:
#Replace empty values with NaN
df2 = df2.replace('s',np.nan)
df3 = df3.replace('s',np.nan)

In [8]:
#Drop rows without without any performance data
df1 = df1.dropna(thresh = 3)
df2 = df2.dropna(thresh = 3)
df3 = df3.dropna(thresh = 3)

In [9]:
#Recast column datatypes to allow for numerical computation
df2['Number of Exams with scores 3 4 or 5'] = df2['Number of Exams with scores 3 4 or 5'].astype(dtype='float64')
df2['Total Exams Taken'] = df2['Total Exams Taken'].astype(dtype='float64')

df3['Critical Reading Mean'] = df3['Critical Reading Mean'].astype(dtype='float64')
df3['Mathematics Mean'] = df3['Mathematics Mean'].astype(dtype='float64')
df3['Writing Mean'] = df3['Writing Mean'].astype(dtype='float64')

In [10]:
#Create new columns providing statistical evaluation for each school
df2['AP_Percentile'] = (df2['Number of Exams with scores 3 4 or 5']/df2['Total Exams Taken']).rank(pct=True)
df3['SAT_Percentile'] = (df3['Critical Reading Mean'] + df3['Mathematics Mean'] + df3['Writing Mean']).rank(pct=True)

In [11]:
#Remove unnecessary columns
df1 = df1[['DBN', 'School_Name', 'Overall_Grade', 'Progress_Grade']]
df2 = df2[['DBN', 'School_Name', 'AP_Percentile']]
df3 = df3[['DBN', 'School_Name', 'SAT_Percentile']]

In [12]:
print(df1.head())

      DBN                     School_Name Overall_Grade Progress_Grade
0  01M015       P.S. 015 Roberto Clemente            C               C
1  01M019             P.S. 019 Asher Levy            B               C
2  01M020            P.S. 020 Anna Silver            B               A
3  01M034  P.S. 034 Franklin D. Roosevelt            C               B
4  01M063       P.S. 063 William McKinley            B               B


In [13]:
print(df2.head())

      DBN                     School_Name  AP_Percentile
0  01M448    UNIVERSITY NEIGHBORHOOD H.S.       0.198675
1  01M450          EAST SIDE COMMUNITY HS            NaN
2  01M515             LOWER EASTSIDE PREP       0.976821
3  01M539  NEW EXPLORATIONS SCI,TECH,MATH       0.701987
5  02M298                Pace High School            NaN


In [14]:
print(df3.head())

      DBN                                     School_Name  SAT_Percentile
0  01M292  Henry Street School for International Studies         0.597150
1  01M448            University Neighborhood High School         0.591969
2  01M450                East Side Community High School         0.727979
3  01M458                   SATELLITE ACADEMY FORSYTH ST         0.365285
5  01M515        Lower East Side Preparatory High School         0.467617


In [15]:
#Open/create database 'schools' and drop dataframes into three tables
conn = sqlite3.connect('schools.db')
c = conn.cursor()

df1.to_sql('Progress_Report', conn, if_exists='replace', index = False)
df2.to_sql('AP_Report', conn, if_exists='replace', index = False)
df3.to_sql('SAT_Report', conn, if_exists='replace', index = False)

In [16]:
"""
These statements work as so:

1.) Block 1 & 2 is a full outer join between AP_Report and SAT_Report that inserts the data into
a temporary table to prevent duplicate rows. Although the DBN is the same between AP_Report & SAT_Report,
the school_names are sometimes different, which pervents the union from removing the duplicate rows.
By making DBN a primary key and inserting the IGNORE statement, the duplicate rows are removed

2.) Blocks 3 & 4 is a left join between the temporary table and the Progress_Report. Progress_Report
table includes schools other than High school, so we do not want to do a full outer join to include them.
Query is then dumped into table Highschools

3.) Block 5 Deletes the temporary table and orginal three tables as they are no longer needed

"""

c.executescript("""

CREATE TABLE TMP(
DBN text NOT NULL PRIMARY KEY,
School_Name text NOT NULL,
AP_Percentile real,
SAT_percentile real);

INSERT or IGNORE INTO TMP
SELECT AP_Report.DBN, AP_Report.School_Name, AP_Percentile, SAT_Percentile 
FROM AP_report LEFT JOIN SAT_report 
ON AP_Report.DBN = SAT_Report.DBN 
UNION 
SELECT SAT_Report.DBN, SAT_Report.School_Name, AP_Percentile, SAT_Percentile 
FROM SAT_Report LEFT JOIN AP_Report 
ON AP_Report.DBN = SAT_Report.DBN;

CREATE TABLE Highschools(
DBN text NOT NULL PRIMARY KEY,
School_Name text NOT NULL,
Ap_Percentile REAL,
Sat_Percentile real,
Overall_Grade text,
Progress_Grade text);

INSERT INTO Highschools
SELECT TMP.DBN, TMP.School_Name, TMP.AP_Percentile, TMP.SAT_percentile, Progress_Report.Overall_Grade, Progress_Report.Progress_Grade
FROM 
TMP LEFT JOIN Progress_Report 
ON TMP.DBN = Progress_Report.DBN;

DROP TABLE TMP;
DROP TABLE Progress_Report;
DROP TABLE SAT_Report;
DROP TABLE AP_Report;
""")

In [17]:
#This query selects schools that have AP & SAT percentiles above 80%, or has an overall grade of "A", or has a grade "B" with a progress grade of "A"
values = (.80,.80,'A','B','A',)
c.execute("""

SELECT *
From Highschools
WHERE AP_Percentile > ? AND SAT_Percentile > ? or Overall_Grade = ? or (Overall_Grade = ? and Progress_grade = ?)
""",values)

In [18]:
#print query
x=c.fetchall()
for rows in x:
	print(rows)

('02M408', 'PROFESSIONAL PERFORMING ARTS', 0.9105960264900662, 0.9222797927461139, 'B ', 'A')
('02M411', 'BARUCH COLLEGE CAMPUS HS', 0.9205298013245033, 0.9572538860103627, None, None)
('02M412', 'NYC LAB HS FOR COLL. STUDIES', 0.847682119205298, 0.9766839378238342, None, None)
('02M416', 'ELEANOR ROOSEVELT HIGH SCHOOL ', 0.8874172185430463, 0.9715025906735751, None, None)
('02M418', 'Millennium High School', 0.8741721854304636, 0.9507772020725389, None, None)
('02M475', 'STUYVESANT HIGH SCHOOL ', 0.9933774834437086, 1.0, None, None)
('02M519', 'TALENT UNLIMITED', 0.8145695364238411, 0.8730569948186528, None, None)
('02M545', 'Dual Language and Asian Studies High School ', 0.9867549668874173, 0.9196891191709845, None, None)
('03M479', 'BEACON SCHOOL', 0.8807947019867549, 0.966321243523316, None, None)
('03M485', 'FIORELLO H.LAGUARDIA HS', 0.8278145695364238, 0.9624352331606217, None, None)
('04M435', 'MANHATTAN CENTER-SCIENCE&MATH', 0.8013245033112583, 0.9326424870466321, None, None)
(

In [19]:
conn.commit()
conn.close()